# <font color='blue'> Random transmissibility multiplier generator for cells on areas of a grid </font>

This scrip is designed to a read a FIPNUM include and expand it. Currently it writes all the values at one, but in future the idea is to only generated random transimissiblity for specific area based on the FIPNUM value.

The main work is done, the addition of writing on area that have specific FIPNUM value should be easy to implement.

In [ ]:
import os
import numpy as np

In [ ]:
path =      r'Windows path, include would be read from here'
file_name = 'FSC_RESERVES_2020_PROP_FIPNUM.GRDECL'
file_path = r'Windows path, include would be written to here'
file_ext  = '.GRDECL'                           # extension for the file, GRDECL is the usual one

In [ ]:
with open(os.path.join(path, file_name), 'r') as f:  # reading a GRDECL file similar to one to be written later
    base = f.readlines()
    
line_number = 0
flag = 0

for item in base:
    item = item.strip()
    
    if flag == 0 and item[:12] == '--Matrix Box': # marked it to find the grid dimensions
        flag = 0.5
        continue
        
    elif flag == 0.5:                             # finding the dimenstions of the grid
        dimen = item.split(" ")
        flag = 1

    elif item[:6] == 'ENDBOX' and flag == 2:      # if ENDBOX is reached, stops the file, consides the last file
        last_line = line_number
        flag = 3
    
    elif item.find('*') > 0   and flag == 1:      # if * is used, marks the start of the grid data
        first_line = line_number + 1
        flag = 2
    
    line_number += 1                              # counting the number of the lines
    
grid_inc = base[first_line:last_line]             # Useful part of the grid file

X = int(dimen[1])
Y = int(dimen[3])
Z = int(dimen[5])

total_cell_count = X * Y * Z  # this returns the total number of the cells in the model

In [ ]:
grid_init = []
for line in grid_inc:
    line = line.strip()
    grid_item = line.split(" ")
    
    for item in grid_item:
        grid_init.append(item) # This is a list of all the values for the grid, comma separated

In [ ]:
grid_b = [] # value behind asteriks
grid_a = [] # value after asteriks
for item in grid_init:
    ast_loc = item.find('*')
        
    if ast_loc > 0:
        counter = int(item[:ast_loc])
        value   =  int(item[ast_loc+1:])
        grid_b.append(counter)
        grid_a.append(value)
    
    elif item != "/":
        grid.append(1)

In [ ]:
new_grid = []  # new_grid 

for item in grid:
    for i in range (item):
        new_grid.append(1)

In [ ]:
keyword_list = ['MULTX', 'MULTX-', 'MULTY', 'MULTY-'] # keyword would be written for each member of this list
counter = 0

for keyword in keyword_list:
    with open(os.path.join(file_path, keyword + file_ext), 'w+') as f:
        f.write('BOX' + '\n')
        f.write('--Matrix Box' + '\n')
        f.write(' 1 ' + str(X) + ' 1 ' + str(Y) + ' 1 ' + str(Z) + ' /' + '\n' + '\n')
        f.write(keyword + '\n')
        f.write(str(X*Y*Z) + '*' + '1' + ' /' + '\n' + '\n')        
        f.write('ENDBOX' + '\n' + '\n')
        
        f.write('BOX' + '\n')
        f.write('--Fracture Box' + '\n')
        f.write(' 1 ' + str(X) + ' 1 ' + str(Y) + ' ' + str(Z+1) + ' ' + str(Z+Z) + ' /' + '\n' + '\n')
        f.write(keyword + '\n')
        
        for item in new_grid: # all the unpacked grid files
            if counter == 35:
                f.write('\n')
                counter = 0

            else:    
                f.write(str(item) + ' ')
                counter = counter + 1
        
        f.write(' /' + '\n')
        f.write('\n' + 'ENDBOX')